In [1]:
!pip install pandas
!pip install numpy

In [2]:
# IMPORTING LIBS
import pandas as pd
import numpy as np
import random

In [3]:
# IMPORT FILE (BC OF GOOGLE COLLAB)
from google.colab import files
  
# select all datasets
uploaded = files.upload()

Saving data1.csv to data1.csv


In [4]:
# GET DATA FROM CSV
import io
  
df1 = pd.read_csv(io.BytesIO(uploaded['data1.csv']))
df1.index.name = "ID"

In [5]:
# GET THE 25 PLAYERS THAT PLAYED THE MOST

sorted_df1 = df1.sort_values(by = "GP", ascending = False).head(25)

In [7]:
# DEFINING FUNCTIONS FOR HILL CLIMBING SOLUTION
def create_solution(df):
  def solution_for_pos(playerPos):
    return random.choice(df.index[df['Pos'] == playerPos].tolist())

  solution = []

  playerIndex = solution_for_pos("Top")
  solution.append(playerIndex)
  
  playerIndex = solution_for_pos("Jungle")
  solution.append(playerIndex)
  
  playerIndex = solution_for_pos("Middle")
  solution.append(playerIndex)
  
  playerIndex = solution_for_pos("ADC")
  solution.append(playerIndex)
  
  playerIndex = solution_for_pos("Support")
  solution.append(playerIndex)
  
  return solution


def expand_neighborhood(solution, df):
  def raffle(pos):
    indexes = {"Top": 0, "Jungle": 1, "Middle": 2, "ADC": 3, "Support": 4}
    players = df.index[df['Pos'] == pos].tolist()
    playerIndex = players.index(solution[indexes[pos]])
    isNew = False
    while not isNew:
      new_pos = random.choice(players)
      isNew = new_pos != playerIndex
    return new_pos

  solution[0] = raffle("Top")
  solution[1] = raffle("Jungle")
  solution[2] = raffle("Middle")
  solution[3] = raffle("ADC")
  solution[4] = raffle("Support")


def target_fn(solution, df, solutionType, target):
  fit_value = 0
  for i in range(5):
    fit_value += target - df[solutionType][solution[i]]

  return fit_value

  

In [8]:
# DEFINING GAMEPLAY TYPES

# type calculation

# Aggressive
# very high kills, moderate deaths, low assists, moderate farm, very high DPM, low WPM

# Farmer
# low kills, low deaths, moderate assists, very high farm, low DPM, moderate WPM

# Support
# very low kills, moderate deaths, high assists, very low farm, low DPM, very high WPM

normalized_data = sorted_df1.copy()
df_min_max_scaled = sorted_df1.copy()

# normalize data
for column in df_min_max_scaled.columns:
  if (normalized_data[column].dtypes == "int64" or normalized_data[column].dtypes == "float64") and column != "GP":
    df_min_max_scaled[column] = (df_min_max_scaled[column] - df_min_max_scaled[column].min()) / (df_min_max_scaled[column].max() - df_min_max_scaled[column].min())    
  

normalized_data = df_min_max_scaled.copy()

normalized_data['aggressive'] = \
  normalized_data.apply(lambda row: abs(((row['K'] * 5) + ((row['D'] * 3)) + (row['A'] * 2) + (row['CSPM'] * 3) + (row['DPM'] * 5) + (row['WPM'] * 2)) / row['GP']), axis=1)

normalized_data['farmer'] = \
  normalized_data.apply(lambda row: abs(((row['K'] * 2) + ((row['D'] * 2)) + (row['A'] * 3) + (row['CSPM'] * 5) + (row['DPM'] * 2) + (row['WPM'] * 3)) / row['GP']), axis=1)

normalized_data['support'] = \
  normalized_data.apply(lambda row: abs(((row['K'] * 1) + ((row['D'] * 3)) + (row['A'] * 4) + (row['CSPM'] * 1) + (row['DPM'] * 2) + (row['WPM'] * 5)) / row['GP']), axis=1)


In [10]:
# DEFINE TEAM TYPE FOR ALGORITHM EXECUTION
TEAM_TYPE = "aggressive"

In [11]:
# 6 traits divided by GP, 1 being max -> 1 * 6 = 6
TARGET = 6

In [15]:
# RUN ALGORITHM
solution = create_solution(normalized_data)
solution_score = target_fn(solution, normalized_data, TEAM_TYPE, TARGET)

for iteration in range(stop_iterations):
  new_solution = create_solution(normalized_data)
  expand_neighborhood(new_solution, normalized_data)

  score = target_fn(new_solution, normalized_data, TEAM_TYPE, TARGET)
  if score < solution_score:
    solution = new_solution
    solution_score = score

In [16]:
# SEE SOLUTION ARRAY
solution

[24, 4, 25, 40, 10]

In [14]:
# SEE DATA SORTED BY TEAM_TYPE value for empyrical analysis
normalized_data.sort_values(by=TEAM_TYPE, ascending=False)

,Player,Team,Pos,GP,W%,CTR%,K,D,A,KDA,KP,KS%,DTH%,FB%,GD10,XPD10,CSD10,CSPM,CS%P15,DPM,DMG%,EGPM,GOLD%,WPM,WCPM,aggressive,farmer,support
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,
40,Zven,Cloud9,ADC,11,45%,45%,0.428571,0.139535,0.193103,0.448276,67.9%,26.4%,16.8%,18%,0.872211,0.399504,1.000000,0.943182,28.7%,0.987097,27.6%,0.915888,25.3%,0.080537,0.465116,0.988525,0.786112,0.449059
34,Tactical,Team Liquid,ADC,13,54%,54%,0.559524,0.395349,0.275862,0.327586,74.9%,30.3%,21.8%,15%,0.452333,0.915633,0.509091,1.000000,32.1%,1.000000,29.7%,1.000000,27.3%,0.060403,0.488372,0.973553,0.762965,0.473156
25,Nisqy,Cloud9,Middle,11,45%,36%,0.369048,0.325581,0.165517,0.224138,62.3%,23.3%,21.9%,27%,0.387424,0.342432,0.212121,0.818182,23.6%,0.752688,23.0%,0.780374,22.8%,0.114094,0.069767,0.872654,0.711307,0.445626
24,Licorice,Cloud9,Top,11,45%,73%,0.285714,0.255814,0.324138,0.362069,72.3%,18.9%,20.0%,27%,0.066937,0.000000,0.272727,0.784091,25.2%,0.825806,24.0%,0.822430,23.4%,0.107383,0.139535,0.867305,0.722699,0.483848
4,Blaber,Cloud9,Jungle,11,45%,73%,0.452381,0.558140,0.234483,0.172414,73.0%,27.7%,28.4%,27%,0.018256,0.039702,0.290909,0.647727,20.6%,0.481720,16.6%,0.663551,20.6%,0.053691,0.813953,0.805860,0.644331,0.449487
33,Svenskeren,Evil Geniuses,Jungle,11,45%,27%,0.464286,0.534884,0.137931,0.137931,74.6%,32.6%,27.4%,45%,0.657201,0.918114,0.575758,0.625000,18.8%,0.498925,16.8%,0.635514,20.2%,0.026846,0.906977,0.784114,0.601411,0.397976
22,Jensen,Team Liquid,Middle,13,54%,54%,0.416667,0.069767,0.296552,0.637931,69.7%,23.4%,13.5%,15%,0.535497,0.637717,0.357576,0.965909,29.0%,0.862366,27.3%,0.925234,26.0%,0.046980,0.325581,0.783789,0.658288,0.364440
1,Bang,Evil Geniuses,ADC,11,45%,82%,0.154762,0.069767,0.227586,0.396552,65.2%,13.8%,14.6%,18%,0.135903,0.173697,0.236364,0.840909,26.8%,0.774194,24.0%,0.742991,23.2%,0.194631,0.441860,0.747386,0.678967,0.421533
27,PowerOfEvil,FlyQuest,Middle,19,58%,68%,0.666667,0.441860,0.558621,0.517241,74.8%,25.6%,17.8%,26%,0.707911,0.540943,0.484848,0.943182,27.7%,0.980645,30.6%,0.897196,25.5%,0.093960,0.418605,0.720887,0.571158,0.400053
